# Interactive masking tool

This notebook show how to use the interactive masking tool.

In [ ]:
%matplotlib widget
from ess.reduce import MaskingTool
from plopp.data.examples import three_bands
import plopp as pp
import scipp as sc

We generate some fake data:

In [ ]:
da = three_bands()
da

## Two-dimensional masks

We then load it into the `MaskingTool`:

In [ ]:
masking_tool = MaskingTool(da, norm='log')

In [ ]:
r, v, h = masking_tool.controls

r.value = True
r._tool.click(0, 200)
r._tool.click(40, 300)

v.value = True
v._tool.click(100, 150)
v._tool.click(150, 150)
v._tool.click(250, 150)
v._tool.click(300, 150)

h.value = True
h._tool.click(100, 50)
h._tool.click(100, 100)

masking_tool.filename.value = "masks-2d.h5"

In [ ]:
masking_tool

You can either save the masks to a file by filling in the text field and clicking the "Save" button,
or get the masks as a `DataGroup` by calling:

In [ ]:
masking_tool.get_masks()

## One-dimensional masks

The tool also works with one-dimensional data.
In this case, only the vertical span tool is active.

In [ ]:
masking_tool = MaskingTool(da.sum('y'), norm='log')

In [ ]:
v = masking_tool.controls[1]
v.value = True
v._tool.click(100, 150)
v._tool.click(150, 150)
v._tool.click(250, 150)
v._tool.click(300, 150)

masking_tool.filename.value = "masks-1d.h5"

In [ ]:
masking_tool

## Using the masks from the tool

We will now show how we can connect the output of the masking tool to further computation,
applying the masks and processing the data further.

We first begin by setting up the same tool as above (in 2D).

In [ ]:
masking_tool = MaskingTool(da, norm='log')

We then create a node that will listen to the changes in the masking tool and apply the masks onto the data before summing along the vertical dimension.

In [ ]:
def apply_masks(da, trigger_node):
    # Get the current masks
    masks = masking_tool.get_masks()
    # Make a shallow copy and add the masks
    out = da.copy(deep=False)
    for name in masks:
        cond = sc.ones(sizes=da.sizes, dtype=bool)
        for dim, bounds in masks[name].items():
            mids = sc.midpoints(da.coords[dim])
            m = (mids > bounds[0]) & (mids < bounds[1])
            cond = cond & m
        out.masks[name] = cond
    # Reduce data and return
    return out.sum('y')

apply = pp.Node(apply_masks, da=da, trigger_node=masking_tool.masking_node)

fig = pp.plot(apply, norm='log')

We now show the two figures, and changes in the tool should update the reduced result.
Adding a rectangle lowers the signal in the same x range but does not bring it to zero.
Adding a vertical span zeros the data.

In [ ]:
r, v, h = masking_tool.controls

r.value = True
r._tool.click(43, 250)
r._tool.click(124, 76)

v.value = True
v._tool.click(188, 150)
v._tool.click(252, 150)

In [ ]:
pp.widgets.Box([masking_tool.fig, fig])